In [1]:
from solve_rg_eqs import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Definitions for useful functions

In [2]:
def bootstrap_g0_multi(L, g0, kc, imscale_v, final_N=None):
    vs = np.zeros(L)

    Ns = np.arange(2, 4*L, 2)
    if final_N is not None:
        Ns = np.arange(2, final_N+2, 2)
    Ne = Ns[-1]//2
    sols = [None for N in Ns]
    for i, N in enumerate(Ns):
        log('')
        log('Now using {} fermions'.format(N))
        log('')
        n = N//2
        dims = (L, n, n)
        print('Dimensions')
        print(dims)
        # Solving for 2n fermions using extrapolation from previous solution
        if n <= 2:
            force_gs=False
            noise_factors=None
            vars = g0_guess(L, n, n, kc, g0, imscale=imscale_v)
        else:
            # The previous solution matches to roughly the accuracy of the solution
            # for the shared variables
            # noise_factors = 10*er*np.ones(len(vars))
            noise_factors = 10**-6*np.ones(len(vars))
            # But we will still need to try random stuff for the 4 new variables
            noise_factors[n-2:n] = 1
            noise_factors[2*n-2:2*n] = 1
            noise_factors[3*n-2:3*n] = 1
            noise_factors[4*n-2:4*n] = 1
        sol = find_root_multithread(vars, kc, g0, dims, imscale_v,
                                    max_steps=MAX_STEPS_1,
                                    use_k_guess=False,
                                    noise_factors=noise_factors,
                                    force_gs=force_gs,
                                    factor=1.1)
        print(vars)
        vars = sol.x
        er = max(abs(sol.fun))
        log('Error with {} fermions: {}'.format(2*n, er))
        sols[i] = sol
        
        if n%2 == 1:
            # setting up for N divisible by 4 next step
            if n > 1:
                vars = sols[i-1].x # Better to start from last similar case
            n -= 1
            incr = 2
        else:
            incr = 1
        
        if n >= 1:
            es, ws = unpack_vars(vars, n, n)
            vars_guess = g0_guess(L, n+incr, n+incr, kc, g0, imscale=imscale_v)
            esg, wsg = unpack_vars(vars_guess, n+incr, n+incr)
            es = np.append(es, esg[-incr:])
            ws = np.append(ws, wsg[-incr:])
            vars = pack_vars(es, ws)        

    return sols, Ns


def solve_job(prev_vars, kc, g, dims, prev_g):
    sol = root(rgEqs, prev_vars, args=(kc, g, dims), method='lm',
               jac=rg_jac)
    er = np.max(np.abs(sol.fun))
    if np.isnan(sol.fun).any():
        er = 1
    if er > 10**-8:
        gts = np.linspace(prev_g, g, 4)
        for gt in gts:
            sol = root(rgEqs, prev_vars, args=(kc, gt, dims), method='lm',
                       jac=rg_jac)
            prev_vars = sol.x
    return sol


def solve_job_q(prev_vars, kc, q, dims, prev_q):
    sol = root(rgEqs_q, prev_vars, args=(kc, q, dims), method='lm')
    er = np.max(np.abs(sol.fun))
    if np.isnan(sol.fun).any():
        er = 1
    if er > 10**-8:
        qts = np.linspace(prev_q, q, 4)
        for qt in qts:
            sol = root(rgEqs_q, prev_vars, args=(kc, qt, dims), method='lm', jac=rg_jac_q)
            prev_vars = sol.x
    return sol


def solve_g_all(l, Ns, g, kc, prev_g, prev_sols):
    dims = [(l, N//2, N//2) for N in Ns]
    with concurrent.futures.ProcessPoolExecutor(max_workers=CPUS) as executor:
        future_results = [executor.submit(solve_job, ps.x, kc, g, dims[Ni], prev_g) for Ni, ps in enumerate(prev_sols)]
        concurrent.futures.wait(future_results)
        for res in future_results:
            try:
                yield res.result()
            except Exception as e:
                print(e)
                
def solve_q_all(l, Ns, g, kc, prev_q, prev_sols):
    dims = [(l, N//2, N//2) for N in Ns]
    with concurrent.futures.ProcessPoolExecutor(max_workers=CPUS) as executor:
        future_results = [executor.submit(solve_job_q, ps.x, kc, q, dims[Ni], prev_q) for Ni, ps in enumerate(prev_sols)]
        concurrent.futures.wait(future_results)
        for res in future_results:
            try:
                yield res.result()
            except Exception as e:
                print(e)

# Code to check gaps for the same N for multiple system sizes

In [ ]:
# ls = 2**np.arange(2, 6)
ls = [3, 4, 8, 16, 24]
print(ls)
Ns = [2, 4, 6, 8]
important_G_rats = np.array([0.25, 0.5, 0.75, 0.9, 1.1, 1.25])
real_vars = {l: {N: {Gr: None for Gr in important_G_rats} for N in Ns} for l in ls}
real_ers = {l: {N: {Gr: None for Gr in important_G_rats} for N in Ns} for l in ls}
energies ={l: {N: {Gr: None for Gr in important_G_rats} for N in Ns} for l in ls}

for i, l in enumerate(ls):
    try:
        print('')
        print('l = {}'.format(l))
        # dg = .01/Ns[-1]
        dg = min(0.01/Ns[-1], 0.005/l)
        g0 = .1*dg
        imk = dg
        imv = .1*g0 

        k = np.arange(1, 2*l+1, 2)*0.5*np.pi/l
        kc = np.concatenate((k, imk*(-1)**np.arange(l)))
        Gc = 1./np.sum(k)
        important_gs = G_to_g(important_G_rats*Gc, k)
        important_qs = 1/important_gs

        print('Getting initial solution at g0 = {}'.format(g0))
        sols, Ns = bootstrap_g0_multi(l, g0, kc, imscale_v=imv, final_N=Ns[-1])
        prev_sols = sols
        ers = np.zeros(len(Ns))
        print('Incrementing g with step size {}'.format(dg))
        g = g0 + dg
        prev_g = g0
        G = g_to_G(g, k)
        important_ind = 0
        keep_going = True
        while keep_going and G/Gc < 0.6: # Tends to fail around .75 Gc
            G = g_to_G(g, k)
            print('G/Gc = {}'.format(G*np.sum(k)))
            prev_sols = sols
            for si, s in enumerate(solve_g_all(l, Ns, g, kc, prev_g, sols)):
                if np.isnan(s.x).any():
                    print('Some of s is nan')
                    print(s.x)
                sols[si] = np.nan_to_num(s) # this isn't great
                ers[si] = np.max(np.abs(s.fun))
                if ers[si] > 10**-7:
                    print('High error at N = {}'.format(Ns[si]))
                    print('Variables here')
                    print(sols[si].x)
            if max(ers) > 10**-7:
                print('Large error: {}. Switching to q'.format(max(ers)))
                print('Errors:')
                print(ers)
                keep_going = False
                # Will use prev_g, prev_sols on restart
            else:
                # saving previous results in case I need to back up
                prev_g = g
                if g + dg > important_gs[important_ind] and g < important_gs[important_ind]:
                    g_imp = important_gs[important_ind]
                    for Ni, s in enumerate(sols):
                        N = Ns[Ni]
                        print('Removing im(k) for N = {}'.format(N))
                        dims = (l, N//2, N//2)
                        vars_r, er_r = increment_im_k(s.x, dims, g_imp, k, kc[l:], steps=5*l, max_steps=10, force_gs=False)
                        print(vars_r)
                        Gr = important_G_rats[important_ind]
                        real_vars[l][N][Gr] = vars_r
                        real_ers[l][N][Gr] = er_r
                    important_ind += 1
                g += dg
        print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print('Finished incrementing g. Now moving to q')
        print('||||||||||||||||||||||||||||||||||||||||')
        print('')
        if not keep_going:
            sols = prev_sols
            g = prev_g
            dq = dg
        else:
            dq = 5*dg
        q = 1/(g + dq)
        prev_q = 1/g
        keep_going = True
        while keep_going and important_ind < len(important_qs):
            G = g_to_G(1/q, k)
            print('G/Gc = {}'.format(G*np.sum(k)))
            for si, s in enumerate(solve_q_all(l, Ns, q, kc, prev_q, sols)):
                if np.isnan(s.x).any():
                    print('Some of s is nan')
                    print(s.x)
                sols[si] = np.nan_to_num(s)
                ers[si] = np.max(np.abs(s.fun))
                if ers[si] > 10**-7:
                    print('High error at N = {}'.format(Ns[si]))
                    print('Variables here')
                    print(sols[si].x)
            if max(ers) > 10**-8:
                print('Errors at q = {}:'.format(q))
                print(ers)
                if dq > .05*dg:
                    dq *= 0.5
                    print('New dq')
                    print(dq)
                    sols = prev_sols
                    q = prev_q + .1*dq
                    print('Stepping back to q = {}'.format(q))
                else:
                    print('Cannot lower dq. Quitting')
                    keep_going = False
            else:
                prev_sols = sols
                if max(ers) < 10**-10:
                    if dq < 50*dg:
                        if G/Gc > 1:
                            dq *= 2
                        else:
                            dq *= 1.5 # can't increase too fast here or we'll fail later.
                        print('Max error: {}'.format(max(ers)))
                        print('New dq')
                        print(dq)
                prev_sols = sols
                prev_q = q
                if q - dq < important_qs[important_ind] and q > important_qs[important_ind]:
                    q_imp = important_qs[important_ind]
                    for Ni, s in enumerate(sols):
                        N = Ns[Ni]
                        print('Removing im(k) for N = {}'.format(N))
                        dims = (l, N//2, N//2)
                        vars_r, er_r = increment_im_k_q(s.x, dims, q_imp, k, kc[l:], steps=5*l)
                        print(vars_r)
                        Gr = important_G_rats[important_ind]
                        real_vars[l][N][Gr] = vars_r
                        real_ers[l][N][Gr] = er_r
                    important_ind += 1
                q -= dq
    except Exception as e:
        print(e)
        print('Failed for l = {}'.format(l))

In [ ]:
print(real_ers[4][8])

In [ ]:
def Z(a,b):
    return a*b/(a-b)

def find_energy(g, ks, es):
    l = len(ks)
    Rs = np.zeros(l)
    for i, k in enumerate(ks):
        Rs[i] = g * np.sum(Z(k, es))
    const = 3*g*np.sum(ks**2)/(1+g*np.sum(ks))
    energy = np.sum(ks*Rs)*2/(1+g*np.sum(ks)) - const
    return energy

In [ ]:
for li, l in enumerate(ls):
    k = np.arange(1, 2*l+1, 2)*0.5*np.pi/l
    Gs = np.array(important_G_rats)/np.sum(k)
    for N in Ns:
        for Gr in important_G_rats:
            dims = (l, N//2, N//2)
            vars_r = real_vars[l][N][Gr]
            if vars_r is not None:
                es, ws = unpack_vars(vars_r, N//2, N//2)
                g = G_to_g(Gr/np.sum(k), k)  
                # print(N)
                # print(g)
                # print(Gr)
                energies[l][N][Gr] = find_energy(g, k, es)
            else:
                print('No variables for l = {}, N = {}, Gr = {}'.format(l, N, Gr))

In [ ]:
for Gr in important_G_rats:
    print(Gr)
    for N in Ns:
        print(N)
        print(energies[4][N][Gr])
plt.figure(figsize=(8, 8))
for l in ls:
    Gr = 1.25
    k = np.arange(1, 2*l+1, 2)*0.5*np.pi/l
    G = Gr/np.sum(k)
    if Ns[-1] == 4*l-2:
        gaps = np.zeros(len(Ns))
        final_ind = -1
    else:
        gaps = np.zeros(len(Ns-1)) # can't fill in the last gap
        final_ind = len(Ns) - 1
    if not np.array([energies[l][N][Gr] is None for N in Ns]).any():
        for Ni, N in enumerate(Ns[:final_ind]):
            if N == 4*l - 2:
                Ep =4*np.sum(k)-3*G*np.sum(k**2)-4*G*(np.sum(k))**2
            else:
                Ep = energies[l][N+2][Gr]
            E = energies[l][N][Gr]
            if N == 2:
                Em = -3*G*np.sum(k**2)
                print('E')
                print(E)
                print('Em')
                print(Em)
            else:
                Em = energies[l][N-2][Gr]
            gaps[Ni] = Ep + Em - 2*E
        plt.plot(Ns,gaps, label='L = {}'.format(2*l))
plt.legend()

In [ ]:
plt.figure(figsize=(4, 4), dpi=300)
plt.axhline(0, color='black')

N = 4
Grs = important_G_rats
for Gr in Grs:
    gaps = np.zeros(len(ls))
    for i, l in enumerate(ls):
        try:
            gaps[i] = energies[l][N+2][Gr] + energies[l][N-2][Gr] - 2*energies[l][N][Gr]
        except:
            pass
    gap_parts = gaps[gaps != 0]
    l_parts = np.array(ls)[gaps != 0]
    plt.plot(.5/np.array(l_parts), gap_parts, label = 'G/Gc = {}'.format(Gr), marker = 'x')
    
plt.ylim(-0.1, 1.5)
plt.ylabel(r'$E(6)+E(2)-2E(4)$')
plt.xlabel('$1/L$')
plt.title('Quartet gaps')
plt.legend()
plt.savefig('Quartet_gaps_scaling_3.png')

In [ ]:
plt.figure(figsize=(4, 4), dpi=300)
N = 4
Gr = 0.5
k = np.arange(1, 2*l+1, 2)*0.5*np.pi/l
G = Gr/np.sum(k)
gaps = np.zeros(len(ls))
for i, l in enumerate(ls):
    gaps[i] = energies[l][N+2][Gr] + energies[l][N-2][Gr] - 2*energies[l][N][Gr]
plt.plot(1./np.array(ls), gaps, label = 'N = {}'.format(N), marker = 'x')
plt.ylim(0, 1)
plt.ylabel(r'E(6)+E(2)-2E(4)$')
plt.xlabel('$l$')
plt.title('Gaps for $G/Gc = 0.25$')

In [ ]:
Grs = important_G_rats
for N in Ns:
    energies_32 = [energies[24][N][Gr] for Gr in Grs]
    plt.plot(Grs, energies_32, label = 'N = {}'.format(N))
plt.legend()
plt.show()

In [ ]:
from exact_diag import ham_op_2, form_basis
l = 4
N = 2
basis = form_basis(2*l, N//2, N//2)
k = np.arange(1, 2*l+1, 2)*0.5*np.pi/l
ees = np.zeros(len(Grs))
ebs = np.zeros(len(Grs))
for Gi, Gr in enumerate(Grs):
    G = Gr/np.sum(k)
    h = ham_op_2(l, G, k, basis)
    e, v = h.eigh()
    print('Closest energy difference')
    E0 = energies[4][N][Gr]
    print(np.min(np.abs(e-E0)))
    print(np.argmin(np.abs(e-E0)))
    ees[Gi] = e[0]
    ebs[Gi] = E0
plt.plot(ees)
plt.plot(ebs)


# 